#### Libs

In [13]:
import pandas as pd

DATADIR = '../datasets/inputs'
DATAOUT = '../datasets/outputs'

geres_cols = {
    '2601 Recife': 'I GERES - Recife',
    '2602 Limoeiro': 'II GERES - Limoeiro',
    '2603 Palmares': 'III GERES - Palmares',
    '2604 Caruaru': 'IV GERES - Caruaru',
    '2605 Garanhuns': 'V GERES - Garanhuns',
    '2606 Arcoverde': 'VI GERES - Arcoverde',
    '2607 Salgueiro': 'VII GERES - Salgueiro',
    '2608 Petrolina': 'VIII GERES - Petrolina',
    '2609 Ouricuri': 'IX GERES - Ouricuri',
    '2610 Afogados da Ingazeira': 'X GERES - Afogados da Ingazeira',
    '2611 Serra Talhada': 'XI GERES - Serra Talhada',
    '2612 Goiana': 'XII GERES - Goiana'
}

#### Dataset de localidade da rede estadual

In [14]:
df_localiz = pd.read_excel(
    f'{DATADIR}/saude_localidade.xlsx', 
    usecols=['codmunres', 'cod_geres', 'geresnome'], 
    dtype={'cod_geres': str}
)
df_localiz.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 187 entries, 0 to 186
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   codmunres  187 non-null    int64 
 1   cod_geres  185 non-null    object
 2   geresnome  185 non-null    object
dtypes: int64(1), object(2)
memory usage: 4.5+ KB


#### 1. Dataset de Partos por Município de residência por Geres estabelecimento

In [15]:
def transform_df(df):
    df.drop('Total', axis=1, inplace=True)
    df['cd_munic'] = df['Munic Residencia PE'].apply(lambda x: x[:6]).astype(int)
    df['municipio'] = df['Munic Residencia PE'].apply(lambda x: x[7:])
    df = df.merge(df_localiz, how='left', left_on='cd_munic', right_on='codmunres')
    return df

In [16]:
df_partos_risco_munic = pd.read_csv(f"{DATADIR}/partos_normal_cesario_risco_munic_resid_geres_ocorr_2020_2022.csv", sep=";")
df_partos_risco_munic = transform_df(df_partos_risco_munic)
df_partos_risco_munic['TIPO_PARTO'] = 'De Risco'

df_partos_normal_munic = pd.read_csv(f"{DATADIR}/partos_normal_cesario_munic_resid_geres_corr_2020_2022.csv", sep=";")
df_partos_normal_munic = transform_df(df_partos_normal_munic)
df_partos_normal_munic['TIPO_PARTO'] = 'Normal/Cesário'

df_partos_munic = pd.concat([df_partos_risco_munic, df_partos_normal_munic])
df_partos_munic.head()

,Munic Residencia PE,2601 Recife,2604 Caruaru,2606 Arcoverde,2608 Petrolina,2609 Ouricuri,cd_munic,municipio,codmunres,cod_geres,geresnome,TIPO_PARTO,2602 Limoeiro,2603 Palmares,2605 Garanhuns,2607 Salgueiro,2610 Afogados da Ingazeira,2611 Serra Talhada,2612 Goiana
0,260005 Abreu e Lima,894,0,0,0,0,260005,Abreu e Lima,260005.0,2601,I GERES - Recife,De Risco,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,260010 Afogados da Ingazeira,75,11,0,0,0,260010,Afogados da Ingazeira,260010.0,2610,X GERES - Afogados da Ingazeira,De Risco,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,260020 Afrânio,0,1,0,84,0,260020,Afrânio,260020.0,2608,VIII GERES - Petrolina,De Risco,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,260030 Agrestina,95,214,0,0,0,260030,Agrestina,260030.0,2604,IV GERES - Caruaru,De Risco,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,260040 Água Preta,185,7,0,0,0,260040,Água Preta,260040.0,2603,III GERES - Palmares,De Risco,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
df_partos_munic.rename(columns=geres_cols, inplace=True)

#### 2. Export

In [18]:
df_partos_munic.to_parquet(f'{DATAOUT}/partos_por_municipio_2019_2022.parquet.gzip')